In [ ]:
import pandas as pd
from itertools import cycle
import gpflow
import numpy as np
from scipy import linalg
from gpflow.utilities import print_summary, positive
from gpflow.ci_utils import ci_niter
from gpflow.optimizers import NaturalGradient
from gpflow import set_trainable
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import statsmodels.api as sm
import timeit

sns.set(style="white")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

np.random.seed(123)

gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_summary_fmt("notebook")

In [ ]:
print("The GPflow version is {0}".format(gpflow.__version__))
print("The tensorflow version is {0}".format(tf.__version__))
print("The tensorflow-probability version is {0}".format(tfp.__version__))

# Data Onboarding and Pre-processing

In [ ]:
mobility_data = pd.read_csv(
    "./Data/Google/Global_Mobility_Report.csv", 
    encoding="ISO-8859-1", 
    parse_dates=["date"],
    dayfirst=True
)

In [ ]:
mobility_data_us = mobility_data[
    (mobility_data["country_region"]=="United States") & (mobility_data["sub_region_1"].isnull())
]
mobility_data_us = mobility_data_us.iloc[:,7:].reset_index(drop=True) # keep only relevant columns
mobility_data_us.iloc[:,1:] = mobility_data_us.iloc[:,1:].apply(lambda x: x/100 + 1)

In [ ]:
mobility_data_us = pd.DataFrame({
    "date": mobility_data_us["date"],
    "HOME": mobility_data_us['residential_percent_change_from_baseline'],
    "WORK": mobility_data_us['workplaces_percent_change_from_baseline'],
    "OTHER": mobility_data_us.iloc[:,1:5].mean(axis=1)
})

In [ ]:
# COVID Cases from Microsoft, includes Recovered and Infected
ms_covid = pd.read_csv(
    "./Data/Microsoft/COVID_CASES.csv", 
    encoding="ISO-8859-1",
    parse_dates=["Updated"],
    dayfirst=False
)
ms_us_covid = ms_covid[(ms_covid["Country_Region"]=="United States") & (ms_covid["AdminRegion1"].isnull())]
ms_us_covid = ms_us_covid.loc[:,["Updated", "Confirmed", "ConfirmedChange", "Deaths", "Recovered"]]
ms_us_covid.reset_index(drop=True, inplace=True)

In [ ]:
# Create the 3 categories S, I, R for US data (UK does not include R)
ms_us_covid = ms_us_covid.fillna(method='ffill').fillna(0)

# DIFF: Instead of smoothing the I, Daily cases and S, ammend the problematic R value @ 2020-07-18
# To match the next 2020-07-19
ms_us_covid.iloc[178, 4] = 1117084.0

ms_us_covid["I"] = ms_us_covid["Confirmed"] - ms_us_covid["Deaths"] - ms_us_covid["Recovered"]

# https://www.kff.org/other/state-indicator/distribution-by-age/?dataView=1&currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D
# 2018; 20<=age
ms_us_covid["S"] = 242620800 - ms_us_covid["I"]

ms_us_covid["index"] = ms_us_covid["Updated"].dt.dayofyear

# DIFF: Create a time variable
# If we have a weekend then assign 0 or weekday 1
ms_us_covid["WEEKDAY"] = 0
ms_us_covid.loc[ms_us_covid.Updated.dt.dayofweek < 5, "WEEKDAY"] = 1

ms_us_covid.dropna(inplace=True)

In [ ]:
final_data = ms_us_covid.set_index("Updated").merge(
    mobility_data_us.set_index("date"), how='left', left_index=True, right_index=True
).reset_index()
final_data.dropna(inplace=True)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20, 4))

final_data.set_index("Updated")[["ConfirmedChange"]].plot(ax=axs[0, 0])
final_data.set_index("Updated")[["I"]].plot(ax=axs[0, 1])
final_data.set_index("Updated")[["S"]].plot(ax=axs[0, 2])
final_data.set_index("Updated")[["HOME"]].plot(ax=axs[1, 0])
final_data.set_index("Updated")[["WORK"]].plot(ax=axs[1, 1])
final_data.set_index("Updated")[["OTHER"]].plot(ax=axs[1, 2])

# Gaussian Process Model

In [ ]:
from experiment import run_experiment, Report

In [ ]:
def train_test_split_func(t, X, y):
    t_train, t_test = t[:-21,:], t[-21:,:]
    X_train, X_test = X[:-21,:], X[-21:,:]
    y_train, y_test = y[:-21,:], y[-21:,:]
    
    return t_train, t_test, X_train, X_test, y_train, y_test

In [ ]:
t = final_data[["Updated"]].to_numpy()
y = final_data[["ConfirmedChange"]].to_numpy()
X = np.hstack([
    (final_data["S"] * final_data["I"]).to_numpy().reshape(-1, 1),
    final_data[["WEEKDAY"]].to_numpy(),
    final_data[["HOME"]].to_numpy(),
    final_data[["WORK"]].to_numpy(),
    final_data[["OTHER"]].to_numpy(),
])

model_params = {
    'kernel': (
        gpflow.kernels.SquaredExponential() *
        gpflow.kernels.Linear() *
        gpflow.kernels.RationalQuadratic()
    ),
    'n_inducing': (15)
}

res, model = run_experiment('$f(S(t)I(t), WKDAY(t), Home(t), Work(t), Other(t)$', 
                            t, X, y, model_params=model_params, train_test_split_func=train_test_split_func, plot=True)
results.add(res)

# Kernel Analysis

In [ ]:
def kernel_analysis(model, model_params, X):
    
    X_sub = X[:-21,:]
    kernel_scaler = preprocessing.StandardScaler().fit(X_sub)
    X_kernel = kernel_scaler.transform(X_sub)
    
    # Extract the eigenvalues
    eigv, _ = linalg.eig(model_params["kernel"](X_kernel)) # pre-opt
    eigv_opt, _ = linalg.eig(model._model.kernel(X_kernel)) # after-opt
    
    # Generate the plots
    fig, ax = plt.subplots(1, figsize=(20, 4))
    fig.suptitle("Before Optimization", fontsize=16)
    ax.plot(np.mean(np.random.multivariate_normal(np.zeros(X_kernel.shape[0]), model_params["kernel"](X_kernel), 100).T, axis=1))
    ax.imshow(k(X_kernel))
    ax.bar(np.arange(40), eigv[0:40])

    fig, ax = plt.subplots(1, figsize=(20, 4))
    fig.suptitle("After Optimization", fontsize=16)
    ax.plot(np.mean(np.random.multivariate_normal(np.zeros(X_kernel.shape[0]), model._model.kernel(X_kernel), 100).T, axis=1))
    ax.imshow(model._model.kernel(X_kernel))
    ax.bar(np.arange(40), eigv_opt[0:40])